In [1]:
from transformers import AutoTokenizer, AutoModel, pipeline
import tensorflow_hub as hub

import pandas as pd
import numpy as np 
import pickle

## Read Data

In [2]:
ekg_denoised = pd.read_pickle('/home/sanjaycollege15/PredictingDiagnoses/Data/ekg_denoised_v2.pkl')
ekg_denoised = ekg_denoised.sample(frac=1, random_state=10)


In [3]:
ekg_denoised.head()

,ICD9_CODE,TEXT
7423,3,sinus rhythm. the p-r interval is prolonged. t...
60592,2,sinus bradycardia. lead v3 was not obtained. c...
62820,2,sinus rhythm consider left atrial abnormality ...
38047,1,sinus rhythm with borderline first degree a-v ...
43024,1,sinus rhythm. non-specific lateral st-t wave a...


## CORe

### Set up Model Pipeline

In [4]:
ekg_denoised['length'] = ekg_denoised.TEXT.str.len()

In [5]:
ekg_denoised['length'].mean()

209.00457612202993

In [6]:
ekg_denoised['length'].describe()

count    68180.000000
mean       209.004576
std        101.165786
min          1.000000
25%        134.000000
50%        190.000000
75%        264.000000
max       1149.000000
Name: length, dtype: float64

In [7]:
tokenizer = AutoTokenizer.from_pretrained("bvanaken/CORe-clinical-outcome-biobert-v1", model_max_length=264)
model = AutoModel.from_pretrained("bvanaken/CORe-clinical-outcome-biobert-v1")

In [8]:
pipe = pipeline('feature-extraction', model=model, tokenizer=tokenizer)

### Embedding Train and Test 

#### Train

In [9]:
features = pipe(ekg_denoised.TEXT[22500:30000].tolist(), 
                padding='max_length',
                truncation=True
               )

In [10]:
features_np = np.squeeze(features)
del features

In [11]:
features_np.shape

(7500, 264, 768)

In [12]:
labels_np = np.array(ekg_denoised.ICD9_CODE[22500:30000])

### Save off Outputs

In [13]:
np.save('/home/sanjaycollege15/PredictingDiagnoses/Data/ekg_denoised_CORe_embeddings_pt4',
        features_np,
        allow_pickle=True)
    
np.save('/home/sanjaycollege15/PredictingDiagnoses/Data/ekg_denoised_CORe_labels_pt4',
        labels_np,
        allow_pickle=True)